# Unwrap to sphere

In [ ]:
from skimage import io, data
import numpy as np
import os
import vedo
import pandas as pd

import napari_segment_blobs_and_things_with_membranes as nsbatwm
import napari_process_points_and_surfaces as nppas
from napari_shape_odyssey import unwrapping
from skimage.data import download_all
import napari

from scipy.spatial import Delaunay

from napari_stress import measurements
from scipy.interpolate import RegularGridInterpolator as rgi
import pyclesperanto_prototype as cle

In [2]:
viewer = napari.Viewer()

Invalid schema for package 'napari-stl-exporter', please run 'npe2 validate napari-stl-exporter' to check for manifest errors.


In [3]:
image = data.cells3d()
nuclei = image[:, 1]
membrane = image[:, 0]

viewer.add_image(membrane, name="membrane")
viewer.add_image(nuclei, name="nuclei")

<Image layer 'nuclei' at 0x1a8a20285e0>

In [4]:
segmentation = nsbatwm.voronoi_otsu_labeling(nuclei, 10, 2)
surface = nppas.label_to_surface(segmentation, 23)
surface_filled = nppas.fill_holes_in_surface(surface)
surface_smoothed = nppas.smooth_surface(surface_filled)
surface_intensity = measurements.measure_intensity_on_surface(surface_smoothed, nuclei, measurement_range=2)
surface_intensity_mean = surface_intensity.mean(axis=1)

surface_intensity = (surface_smoothed[0], surface_smoothed[1], surface_intensity_mean.values)

In [5]:
sphere_intensity = unwrapping.map_surface_to_sphere(surface_intensity)
unwrapping.mercator_projection(sphere_intensity)

ValueError: cannot convert float NaN to integer

In [6]:
layer_surf = viewer.add_surface(
    sphere_intensity,
    colormap='inferno', opacity=0.5, name='surface')
layer_surf.wireframe.visible = True